In [1]:
from glob import glob

import pandas as pd
from tqdm import tqdm

import harmonic_inference.utils.harmonic_utils as hu
import harmonic_inference.utils.eval_utils as eu
from harmonic_inference.data.data_types import PitchType, ChordType, NO_REDUCTION, TRIAD_REDUCTION

In [2]:
for df_path in tqdm(glob("output_sps*/**/*_results.tsv", recursive=True)):
    results_df = pd.read_csv(df_path, sep="\t", index_col=0)
    results_df.loc[:, ["gt_chord_root", "gt_chord_type", "gt_chord_inv", "est_chord_root", "est_chord_type", "est_chord_inv", "root_correct", "triad_correct", "7th_correct", "inv_correct", "full_correct"]] = 0

    for row_id, row in results_df.iterrows():
        for prefix in ["gt", "est"]:
            root_string, inv_string = row[f"{prefix}_chord"].split(", ")
            root_string, chord_type_string = root_string.split(":")

            root = hu.get_pitch_from_string(root_string, PitchType.MIDI)
            chord_type = hu.get_chord_type_from_string(chord_type_string)
            inv = int(inv_string[-1])
            
            results_df.loc[row_id, [f"{prefix}_chord_root", f"{prefix}_chord_type", f"{prefix}_chord_inv"]] = [root, chord_type, inv]

        results_df.loc[row_id, "root_correct"] = 0 == eu.get_chord_distance(
            results_df.loc[row_id, "gt_chord_root"], ChordType.MAJOR, 0,
            results_df.loc[row_id, "est_chord_root"], ChordType.MAJOR, 0,
        )
        results_df.loc[row_id, "triad_correct"] = 0 == eu.get_chord_distance(
            results_df.loc[row_id, "gt_chord_root"], results_df.loc[row_id, "gt_chord_type"], 0,
            results_df.loc[row_id, "est_chord_root"], results_df.loc[row_id, "est_chord_type"], 0,
            reduction=TRIAD_REDUCTION,
        )
        results_df.loc[row_id, "7th_correct"] = 0 == eu.get_chord_distance(
            results_df.loc[row_id, "gt_chord_root"], results_df.loc[row_id, "gt_chord_type"], 0,
            results_df.loc[row_id, "est_chord_root"], results_df.loc[row_id, "est_chord_type"], 0,
        )
        results_df.loc[row_id, "inv_correct"] = 0 == eu.get_chord_distance(
            results_df.loc[row_id, "gt_chord_root"], ChordType.MAJOR, results_df.loc[row_id, "gt_chord_inv"],
            results_df.loc[row_id, "est_chord_root"], ChordType.MAJOR, results_df.loc[row_id, "est_chord_inv"],
        )
        results_df.loc[row_id, "full_correct"] = 0 == eu.get_chord_distance(
            results_df.loc[row_id, "gt_chord_root"], results_df.loc[row_id, "gt_chord_type"], results_df.loc[row_id, "gt_chord_inv"],
            results_df.loc[row_id, "est_chord_root"], results_df.loc[row_id, "est_chord_type"], results_df.loc[row_id, "est_chord_inv"],
        )

    results_df.to_csv(df_path, sep="\t")

100%|██████████| 310/310 [09:50<00:00,  1.90s/it]
